We first update the environment variables with AI Core keys Copy the keys from this link - https://sap.sharepoint.com/:f:/r/teams/AIDeveloperFoundationProgram/Shared%20Documents/India%20-%20April%202024%20Cohort%201/Keys/IES_GEN_AI_XL-Keys?csf=1&web=1&e=zq9rHP

HANA VECTOR USER, PASS & HOST information, you can find from the week-3 readme file
https://github.tools.sap/AILearning/AIDeveloperFoundationProgram/blob/main/week-3/README.md

"HANA_VECTOR_USER" - Administrator User (DBADMIN)

"HANA_VECTOR_PASS" - Administrator Password provided during setup

"HANA_HOST_VECTOR" - SQL Endpoint (remove :443 while copying from endpoint url)

In [13]:
import os 

# Define Keys HERE
env_vars = {   
  "HANA_VECTOR_USER": "DBADMIN",
  "HANA_VECTOR_PASS": "Welcome2!",
  "HANA_HOST_VECTOR": "04a2209c-bf66-479a-bab5-2ebe5231747b.hana.trial-us10.hanacloud.ondemand.com",
 "AICORE_AUTH_URL": "https://sli-md-office-fyhziaa8.authentication.eu12.hana.ondemand.com/oauth/token",
 "AICORE_CLIENT_ID": "sb-64e38075-b7db-450d-9dd4-9930770a426d!b374611|xsuaa_std!b318061",
 "AICORE_CLIENT_SECRET": "373d7026-c9ed-4515-ab94-c4052bee17d0$_SpyGk4MTrxQ7TmGTxRkyX94deNKx9wFcuK_HpmLIPI=",
 "AICORE_RESOURCE_GROUP": "default",
 "AICORE_BASE_URL":    "https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2"
}

os.environ.update(env_vars)

In [ ]:
DBADMIN",
  "HANA_VECTOR_PASS": "Welcome2!",
  "HANA_HOST_VECTOR": "04a2209c-bf66-479a-bab5-2ebe5231747b.hana.trial-us10.hanacloud.ondemand.com

In [2]:
!pip install hdbcli

In [3]:
%pip install hana_ml

Note: you may need to restart the kernel to use updated packages.


use pip install hdbcli & pip install hana_ml to import these libraries (or pip3 install..)

In [4]:
# Import dbapi from hdbcli library
from hdbcli import dbapi
import hana_ml.dataframe as dataframe

The below code retrieves the values of HANA user, Password & Host from environment variables. You can directly pass the same as well instead of first adding to the environment variable

In [5]:
# Get the HANA Cloud username from environment variables
HANA_USER_VDB = os.getenv('HANA_VECTOR_USER')
# Get the HANA Cloud password from environment variables
HANA_PASSWORD_VDB = os.getenv('HANA_VECTOR_PASS')
# Get the HANA Cloud host from environment variables
HANA_HOST  = os.getenv('HANA_HOST_VECTOR')

Establish the connection to HANA DB

In [12]:
# Use connection settings from the environment
connection = dbapi.connect(
    address=HANA_HOST,
    port=30015,
    user=HANA_USER_VDB,
    password=HANA_PASSWORD_VDB,
    encrypt='true',
    autocommit=True
)

Error: (-10709, 'Connection failed (RTE:[89009] Socket connect timeout (20000 ms); $host$= (192.168.1.101:57501 -> d7a4556b-17e2-4ab0-b2bd-9bde685822a7.hana.prod-eu12.hanacloud.ondemand.com:30015))')

Connection Context in this notebook is used to validate the hana version

In [ ]:

conn = dataframe.ConnectionContext(
    address=HANA_HOST,  
    port=443,
    user=HANA_USER_VDB,
    password=HANA_PASSWORD_VDB,
    encrypt='true',
    autocommit=True
)

Error: (-10709, 'Connection failed (RTE:[89013] Socket closed by peer {192.168.1.101:54274 -> 35.156.35.115:443} (192.168.1.101:54274 -> d7a4556b-17e2-4ab0-b2bd-9bde685822a7.hana.prod-eu12.hanacloud.ondemand.com:443))')

If the HANA version is <4, then upgrade of the instance in your BTP account is required

In [15]:
conn.hana_version()

NameError: name 'conn' is not defined

Import necessary langchain modules along with assignment of the proxy client as gen-ai-hub & chat_llm model initiation as gpt-35-turbo, which can be changed based on model availability in GENAI Hub

In [ ]:
from langchain.prompts.chat import (
        AIMessagePromptTemplate,
        ChatPromptTemplate,
        HumanMessagePromptTemplate,
        SystemMessagePromptTemplate,
    )

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
chat_llm = ChatOpenAI(proxy_model_name='gpt-35-turbo', proxy_client=proxy_client, temperature=0.0)

This code performs the document split (loaded using PyPDF Loader available with Langchain) in chunks of size 1000 & chunk overlap 50. If you wish to use a text file, you can use TextLoader added as import below (for other file types supported, refer - https://python.langchain.com/docs/modules/data_connection/document_loaders/)

split of the text is done and embedding model is initiated using text-embedding-ada-002


In [ ]:
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores.hanavector import HanaDB

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
loader = PyPDFLoader('./flexben_2023.pdf')
pages = loader.load_and_split(text_splitter)
documents = loader.load()

texts = text_splitter.split_documents(documents)

embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

Initiate the vector db adding the embedding model, connection & provide the table name. If the table doesn't exist, a table will be created automatically
if you wish to remove the previous vector entries (important when u re-run this code), use db.delete & then add the embeddings using db.add_documents

In [ ]:
db = HanaDB(
    embedding=embedding_model, connection=connection, table_name="FLEXBEN_POLICY"
)

# Delete already existing documents from the table
db.delete(filter={})

# add the loaded document chunks
db.add_documents(texts)

This code is used to retrieve the embeddings data from HANA DB

In [ ]:
# take a look at the table
hdf = conn.sql(''' SELECT "VEC_TEXT", "VEC_META", TO_NVARCHAR("VEC_VECTOR") AS "VEC_VECTOR" FROM "FLEXBEN_POLICY" ''')
df = hdf.head(5).collect()
df

Based on the query, you can now perform similarity search & you can filter with top k most similar chunks

In [ ]:
query = "What is Flexben?"
docs = db.similarity_search(query, k=2)

for doc in docs:
    print("-" * 80)
    print(doc.page_content)

Initiate the retriever from db & also retrieval QA with model & retriever

In [ ]:
retriever = db.as_retriever()

qa = RetrievalQA.from_llm(
    llm=chat_llm, retriever=retriever)  # pass the BTP OpenAI model

Now you can ask any queries and using qa.invoke, the response will be given as output

In [ ]:
query = "What is flexben?"
result = qa.invoke(query)

In [ ]:
result

In [ ]:
query = "How many flexben points do i get?"
result = qa.invoke(query)

In [ ]:
result

## Additional Exploration - LLM Chain

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
You are an expert in SAP Technologies. You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end.

```
{context}
```

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


llm = chat_llm
memory = ConversationBufferMemory(
    memory_key="chat_history", output_key="answer", return_messages=True
)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    memory=memory,
    verbose=True,
    combine_docs_chain_kwargs={"prompt": PROMPT},
)

In [ ]:
question = "What is flexben?"

result = qa_chain.invoke({"question": question})
print("Answer from LLM:")
print("================")
print(result["answer"])

source_docs = result["source_documents"]
print("================")
print(f"Number of used source document chunks: {len(source_docs)}")

In [ ]:
for doc in result["source_documents"]:
    print("-" * 80)
    print(doc.page_content)
    print(doc.metadata)

## Reference:

https://python.langchain.com/docs/integrations/vectorstores/sap_hanavector/

https://github.tools.sap/Artificial-Intelligence-CoE/ies-genai-platform-models-cookbook/tree/main/examples/python/SAP-HANA-Cloud-VectorEngine-PoC

https://github.wdf.sap.corp/hana-multi-model/vector-getting-started

https://discovery-center.cloud.sap/protected/index.html#/mymissiondetail/95888/card/10571818/?tab=projectboard